In [24]:
import os
import numpy as np
import pandas as pd

# 특정 파일에서 covariance matrix 데이터를 읽어들이는 함수
def read_covariance_matrix(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    
    orientation_cov = []
    angular_velocity_cov = []
    linear_acceleration_cov = []
    
    current_matrix = None
    for line in lines:
        if "Orientation covariance matrix" in line:
            current_matrix = orientation_cov
        elif "Angular velocity covariance matrix" in line:
            current_matrix = angular_velocity_cov
        elif "Linear acceleration covariance matrix" in line:
            current_matrix = linear_acceleration_cov
        elif line.strip():  # 빈 줄이 아닌 경우
            values = list(map(float, line.split()))
            current_matrix.append(values)
    
    return np.array(orientation_cov), np.array(angular_velocity_cov), np.array(linear_acceleration_cov)

# 폴더 내 모든 covariance matrix 파일을 엑셀로 저장하는 함수
def save_all_covariances_to_excel(folder_path, output_excel_path):
    # 엑셀 파일을 쓰기 모드로 열기
    with pd.ExcelWriter(output_excel_path, engine='openpyxl') as writer:
        sheet_added = False  # 하나 이상의 시트가 추가되었는지 확인
        
        # 지정된 폴더 내 모든 파일 검색
        for filename in os.listdir(folder_path):
            # 'covariance_matrix'로 시작하고 .txt 확장자를 가진 파일만 처리
            if filename.startswith("covatiance_matrix_-") and filename.endswith(".txt"):
                file_path = os.path.join(folder_path, filename)
                
                # 파일에서 covariance matrix 읽기
                orientation_cov, angular_velocity_cov, linear_acceleration_cov = read_covariance_matrix(file_path)
                
                # 각 행렬을 데이터프레임으로 변환
                orientation_df = pd.DataFrame(orientation_cov, columns=['X', 'Y', 'Z'])
                angular_velocity_df = pd.DataFrame(angular_velocity_cov, columns=['X', 'Y', 'Z'])
                linear_acceleration_df = pd.DataFrame(linear_acceleration_cov, columns=['X', 'Y', 'Z'])
                
                # 파일 이름을 기반으로 시트 이름 설정
                sheet_prefix = filename.replace(".txt", "")
                
                # 각 행렬을 개별 시트에 저장
                orientation_df.to_excel(writer, sheet_name=f'{sheet_prefix}_Orientation', index=False)
                angular_velocity_df.to_excel(writer, sheet_name=f'{sheet_prefix}_AngularVelocity', index=False)
                linear_acceleration_df.to_excel(writer, sheet_name=f'{sheet_prefix}_LinearAcceleration', index=False)
                
                sheet_added = True  # 시트가 추가되었음을 표시
                
        # 파일에 시트를 추가하지 못한 경우 기본 시트 추가
        if not sheet_added:
            df_empty = pd.DataFrame({"Message": ["No covariance matrix files found."]})
            df_empty.to_excel(writer, sheet_name='No Data', index=False)

# 폴더 경로 및 출력 엑셀 파일 경로 설정
folder_path = '/home/hys/Downloads/imu_covariance_matrix'
output_excel_path = 'all_covariance_matrices.xlsx'

# 모든 covariance matrix 파일을 엑셀로 저장
save_all_covariances_to_excel(folder_path, output_excel_path)
print(f"All covariance matrices saved to {output_excel_path}")


All covariance matrices saved to all_covariance_matrices.xlsx
